In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator and then re-execute this cell.')
else:
    print(gpu_info)

In [ ]:
from psutil import virtual_memory

ram = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM.'.format(ram))
if ram < 20:
    print('To enable a high-RAM runtime, select the Runtime > "Change runtime type", then select High-RAM in the Runtime shape dropdown '
          'and then re-execute this cell.')
else:
    print('You are using a high-RAM runtime.')

In [ ]:
from tensorflow import config

physical_devices = config.list_physical_devices('GPU')
try:
    config.experimental.set_memory_growth(physical_devices[0], True)
except Exception as exception:
    print(exception)

In [ ]:
!pip install tensorflow-addons

In [ ]:
from tensorflow import device
from tensorflow_addons import layers as new_layers
from tensorflow.keras import losses
from tensorflow.keras import models

import csv
import gc
import h5py
import imageio
import numpy as np
import os

In [ ]:
town = 'Moscow'  #@param ['Berlin', 'Istanbul', 'Moscow']

model_path = '/content/gdrive/My Drive/Licenta/Traffic4Cast/{}/checkpoints/UNet3/model_39.h5'.format(town)
with device('gpu:0'):
    model = models.load_model(model_path)
loss = losses.MeanSquaredError()

validation_files = '/content/gdrive/My Drive/Licenta/Traffic4Cast/{}/files/validation'.format(town)
testing_files = '/content/gdrive/My Drive/Licenta/Traffic4Cast/{}/files/testing'.format(town)

validation_logs = '/content/gdrive/My Drive/Licenta/Traffic4Cast/{}/logs/UNet3/validation/logs.csv'.format(town)
testing_logs = '/content/gdrive/My Drive/Licenta/Traffic4Cast/{}/logs/UNet3/testing/logs.csv'.format(town)

validation_movies = '/content/gdrive/My Drive/Licenta/Traffic4Cast/{}/movies/validation/UNet3'.format(town)
testing_movies = '/content/gdrive/My Drive/Licenta/Traffic4Cast/{}/movies/testing/UNet3'.format(town)

movie_names = ['ne_volume', 'ne_average_speed', 'se_volume', 'se_average_speed', 
               'sv_volume', 'sv_average_speed', 'nv_volume', 'nv_average_speed']

In [ ]:
def get_file_names(files):
    return os.listdir(files)

In [ ]:
def get_data(file_path):
    file = h5py.File(file_path, 'r')
    group_key = list(file.keys())[0]
    data = np.array(file[group_key][:], dtype=np.float32)  # (288, 495, 436, 9) for validation or
                                                           # (3, 12, 485, 436, 9) or
                                                           # (4, 12, 485, 436, 9) for testing
    file.close()
    return data

In [ ]:
def get_validation_data(data):
    data = np.take(data, np.arange(8), axis=-1)  # keep only the dynamic channels
    data = np.array(np.split(data, 48))  # split in 48 batches of 3 + 3 timestamps
    data = np.moveaxis(data, 1, -1).reshape((48, 495, 436, -1))  # combine the timestamps with the channels
    data /= 255.0
    inputs = data[:, :, :, :24]
    outputs = data[:, :, :, 24:]
    return inputs, outputs

In [ ]:
def get_testing_data(data):
    data = np.take(data, np.arange(8), axis=-1)  # keep only the dynamic channels
    data = np.array(np.split(data, 2, axis=1))  # split in 2 batches of 3 + 3 timestamps
    data = np.moveaxis(data, 2, -1).reshape((data.shape[0], 2, 495, 436, -1))  # combine the timestamps with the channels
    data = data.reshape((-1, 495, 436, 48))  # combine the predictions with the batches
    data /= 255.0
    inputs = data[:, :, :, :24]
    outputs = data[:, :, :, 24:]
    return inputs, outputs

In [ ]:
def get_restored_validation_data(data):
    data *= 255.0
    data = np.moveaxis(data.reshape((48, 495, 436, 8, 3)), -1, 1)  # (48, 3, 495, 436, 8)
    return data.reshape((-1, 495, 436, 8))  # (144, 495, 436, 8)

In [ ]:
def get_restored_testing_data(data):
    data *= 255.0
    data = np.moveaxis(data.reshape((data.shape[0], 495, 436, 8, 3)), -1, 1)  # (6, 3, 495, 436, 8) or
                                                                              # (8, 3, 495, 436, 8)
    return data.reshape((-1, 495, 436, 8))  # (18, 495, 436, 8) or (24, 495, 436, 8)

In [ ]:
def get_validation_mask(data):
    data = np.array(np.split(data, 48))  # split in 48 batches of 3 + 3 timestamps
    data = data[:, :3, :, :, :]  # keep only the inputs
    data = data.reshape(-1, 495, 436, 9)
    data = np.take(data, [1, 3, 5, 7], axis=-1)  # take only the average speed channels
    data = np.moveaxis(data, -1, 1).reshape(-1, 495, 436)  # (144 * 4, 495, 436)
    data = np.mean(data, axis=0, dtype=np.float32)
    return (data > 0.0).astype(np.uint8)

In [ ]:
def get_testing_mask(data):
    data = np.array(np.split(data, 2, axis=1))  # split in 2 batches of 3 + 3 timestamps
    data = data[:, :, :3, :, :, :]  # keep only the inputs
    data = data.reshape(-1, 495, 436, 9)
    data = np.take(data, [1, 3, 5, 7], axis=-1)  # take only the average speed channels
    data = np.moveaxis(data, -1, 1).reshape(-1, 495, 436)  # (18 * 4, 495, 436) or (24 * 4, 495, 436)
    data = np.mean(data, axis=0, dtype=np.float32)
    return (data > 0.0).astype(np.uint8)

In [ ]:
def create_validation_movie(actual, prediction, mask, movies, file_path):
    actual = np.sum(actual, axis=-1, dtype=np.uint8)
    prediction = np.sum(prediction, axis=-1, dtype=np.uint8)
    prediction *= mask
    frames = np.concatenate((actual, prediction), axis=-1)
    path = os.path.join(movies, os.path.basename(file_path)[:-7] + '.gif')
    imageio.mimsave(path, [frame for frame in frames], fps=3)
    print('created movie:', path)

In [ ]:
def create_testing_movie(actual, prediction, mask, movies, file_path):
    actual = np.sum(actual, axis=-1, dtype=np.uint8)
    prediction = np.sum(prediction, axis=-1, dtype=np.uint8)
    prediction *= mask
    frames = np.concatenate((actual, prediction), axis=-1)
    path = os.path.join(movies, os.path.basename(file_path)[:-3] + '.gif')
    imageio.mimsave(path, [frame for frame in frames], fps=3)
    print('created movie:', path)

In [ ]:
def create_validation_movies(actual, prediction, mask, movies, file_path):
    for channel in range(8):
        first = np.squeeze(actual[:, :, :, channel])
        second = np.squeeze(prediction[:, :, :, channel])
        second *= mask
        frames = np.concatenate((first, second), axis=-1)
        frames = np.clip(frames * 3.0, 0.0, 255.0).astype('uint8')
        path = os.path.join(movies, os.path.basename(file_path)[:-6] + movie_names[channel] + '.gif')
        imageio.mimsave(path, [frame for frame in frames], fps=3)
        print('created movie:', path)

In [ ]:
def create_testing_movies(actual, prediction, mask, movies, file_path):
    for channel in range(8):
        first = np.squeeze(actual[:, :, :, channel])
        second = np.squeeze(prediction[:, :, :, channel])
        second *= mask
        frames = np.concatenate((first, second), axis=-1)
        frames = np.clip(frames * 3.0, 0.0, 255.0).astype('uint8')
        path = os.path.join(movies, os.path.basename(file_path)[:-3] + '_' + movie_names[channel] + '.gif')
        imageio.mimsave(path, [frame for frame in frames], fps=3)
        print('created movie:', path)

In [ ]:
def get_validation_loss(model, loss, file_path, movies):
    data = get_data(file_path)
    inputs, actual = get_validation_data(data)
    with device('gpu:0'):
        prediction = model.predict(inputs)
        prediction = np.clip(prediction, 0.0, 1.0)
    gc.collect()
    return loss(actual, prediction).numpy()

In [ ]:
def get_testing_loss(model, loss, file_path, movies):
    data = get_data(file_path)
    inputs, actual = get_testing_data(data)
    with device('gpu:0'):
        prediction = model.predict(inputs)
        prediction = np.clip(prediction, 0.0, 1.0)
    gc.collect()
    return loss(actual, prediction).numpy()

In [ ]:
validation_file_names = get_file_names(validation_files)

log_file = open(validation_logs, 'w', newline='')
log_writer = csv.writer(log_file)
log_writer.writerow(['file', 'loss'])
log_file.flush()

losses = np.zeros(shape=(len(validation_file_names),), dtype=np.float64)

for index, file_name in enumerate(validation_file_names):
    print('file:', index)
    result = get_validation_loss(model, loss, os.path.join(validation_files, file_name), validation_movies)
    print('validation loss:', result)
    log_writer.writerow([file_name, result])
    log_file.flush()
    losses[index] = result

mean = np.mean(losses)
median = np.median(losses)
print('validation loss mean:', mean)
print('validation loss median:', median)

log_writer.writerow(['mean', mean])
log_writer.writerow(['median', median])
log_file.flush()

log_file.close()

In [ ]:
file_path = os.path.join(validation_files, validation_file_names[np.argmin(losses)])
data = get_data(file_path)
mask = get_validation_mask(data)
inputs, actual = get_validation_data(data)
with device('gpu:0'):
    prediction = model.predict(inputs)
    prediction = np.clip(prediction, 0.0, 1.0)
gc.collect()
actual = get_restored_validation_data(actual)
prediction = get_restored_validation_data(prediction)

In [ ]:
create_validation_movie(actual, prediction, mask, validation_movies, file_path)

In [ ]:
create_validation_movies(actual, prediction, mask, validation_movies, file_path)

In [ ]:
testing_file_names = get_file_names(testing_files)

log_file = open(testing_logs, 'w', newline='')
log_writer = csv.writer(log_file)
log_writer.writerow(['file', 'loss'])
log_file.flush()

losses = np.zeros(shape=(len(testing_file_names),), dtype=np.float64)

for index, file_name in enumerate(testing_file_names):
    print('file:', index)
    result = get_testing_loss(model, loss, os.path.join(testing_files, file_name), testing_movies)
    print('testing loss:', result)
    log_writer.writerow([file_name, result])
    log_file.flush()
    losses[index] = result

mean = np.mean(losses)
median = np.median(losses)
print('testing loss mean:', mean)
print('testing loss median:', median)

log_writer.writerow(['mean', mean])
log_writer.writerow(['median', median])
log_file.flush()

log_file.close()

In [ ]:
file_path = os.path.join(testing_files, testing_file_names[np.argmin(losses)])
data = get_data(file_path)
mask = get_testing_mask(data)
inputs, actual = get_testing_data(data)
with device('gpu:0'):
    prediction = model.predict(inputs)
    prediction = np.clip(prediction, 0.0, 1.0)
gc.collect()
actual = get_restored_testing_data(actual)
prediction = get_restored_testing_data(prediction)

In [ ]:
create_testing_movie(actual, prediction, mask, testing_movies, file_path)

In [ ]:
create_testing_movies(actual, prediction, mask, testing_movies, file_path)